<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Image annotation UI

Open-source annotation tools for object detection and for image segmentation exist, however for image classification we were not able to find a good program. Hence this notebook provides a simple UI to label images. Each image can be annotated with one or multiple classes, or marked as "Exclude" to indicate that the image should not be used for model trainining or evaluation. 

Note that, for single class annotation tasks, one does not need any UI but can instead simply drag&drop images into separate folder for the respective classes. 

In [1]:
# Ensure edits to libraries are loaded and plotting is shown in the notebook.
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [13]:
import sys
sys.path.append("../")
from utils_ic.anno_utils import AnnotationWidget
from utils_ic.datasets import unzip_url, Urls

Set parameters: location of the images to annotate, and path where to save the annotations. Here `unzip_url` is used to download example data if not already present, and set the path.

In [17]:
IM_DIR = os.path.join((unzip_url(Urls.fridge_objects_path, exist_ok=True)), 'can')
ANNO_PATH = "cvbp_ic_annotation.pkl"
print(f"Using images in directory: {IM_DIR}.")

Using images in directory: C:\Users\pabuehle\Desktop\ComputerVisionBestPractices\image_classification\data\fridgeObjects\can.


Start the UI. Check the "Allow multi-class labeling" box to allow that images can be annotated as multiple classes. When in doubt what the annotation for an image should be, or for any other reason (e.g. blur or over-exposure), mark an image as "EXCLUDE". All annotations are saved to (and loaded from) a pandas dataframe with path specified in `anno_path`. 

<center>
<img src="https://cvbp.blob.core.windows.net/public/images/document_images/anno_ui2.jpg" style="width: 600px;"/>
<i>Annotation UI example</i>
</center>

In [16]:
w_anno_ui = AnnotationWidget(
    labels       = ["can", "carton", "milk_bottle", "water_bottle"],
    im_dir       = IM_DIR,
    anno_path    = ANNO_PATH,
    im_filenames = None #Set to None to annotate all images in IM_DIR
)

display(w_anno_ui.show())

Below is an example how to create a fast.ai ImageList object using the ground truth annotations generated by the AnnotationWidget. Note that fast.ai does not support the exclude flag, hence we remove these images before calling fast.ai's `from_df()` and `label_from_df()` functions.

```python
import pandas as pd
from fastai.vision import ImageList,ImageDataBunch

# Load annotation, discard excluded images, and convert to format fastai expects
annos = pd.read_pickle(ANNO_PATH) 
keys = [key for key in annos if (not annos[key].exclude and len(annos[key].labels)>0)]
df = pd.DataFrame([(anno[0], ",".join(anno[1].labels)) for anno in annos[keys].items()], 
                  columns = ["name", "label"])
display(df)

# Create an ImageList and assign labels 
data = (ImageList.from_df(path=IM_DIR, df = df)
       .split_by_rand_pct(valid_pct=0.5)
       .label_from_df(label_delim=','))
```


### Image Scraping and Annotation

Collecting a sufficiently large number of annotated images for training and testing can be difficult. One way to over-come this problem is to scrape images from the Internet. For example, see below the Bing Image Search results for the query "tshirt striped". As expected, most images indeed are striped t-shirts, and the few incorrect or ambiguous images (such as column 1, row 1; or column 3, row 2) can be identified and removed easily:
<p align="center">
<img src="media/bing_search_striped.jpg" alt="alt text" width="600"/>
</p>

To generate a large and diverse dataset, multiple queries should be used. For example 7\*3 = 21 queries can by synthesized automatically using all combinations of 7 clothing items {blouse, hoodie, pullover, sweater, shirt, tshirt, vest} and 3 attributes {striped, dotted, leopard}. Downloading the top 50 images per query would then lead to a maximum of 21*50=1050 images.

Rather than manually downloading images from Bing Image Search, it is much easier to instead use the [Cognitive Services Bing Image Search API](https://www.microsoft.com/cognitive-services/en-us/bing-image-search-api) which returns a set of image URLs given a query string:
<p align="center">
<img src="media/bing_image_search_api.jpg" alt="alt text" width="600"/>
</p>

Some of the downloaded images will be exact or near duplicates (e.g. differ just by image resolution or jpg artifacts) and should be removed so that the training and test split do not contain the same images. This can be achieved using a hashing-based approach which works in two steps: (i) first, the hash string is computed for all images; (ii) in a second pass over the images, only those are kept with a hash string which has not yet been seen. All other images are discarded. We found the *dhash* approach in the Python library *imagehash* and described in this [blog](http://www.hackerfactor.com/blog/index.php?/archives/529-Kind-of-Like-That.html) to perform well, with the parameter `hash_size` set to 16. It is OK to incorrectly remove some non-duplicates images, as long as the majority of the real duplicates get removed.